In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
print(os.listdir("data"))

['test.csv', 'train.csv']


In [120]:
from keras import layers
from keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import np_utils
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.optimizers import Adam, RMSprop
import scipy.misc
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [121]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

train_data.shape, test_data.shape

((42000, 785), (28000, 784))

In [122]:
Y_train = train_data.iloc[:,0]
X_train = train_data.iloc[:,1:]

X_train.shape, Y_train.shape

((42000, 784), (42000,))

In [123]:
X_train = X_train.as_matrix().reshape(X_train.shape[0],28,28,1).astype(np.float32)
test_data = test_data.as_matrix().reshape(test_data.shape[0],28,28,1).astype(np.float32)
Y_train = Y_train.as_matrix().reshape(Y_train.shape[0],1)

X_train.shape, Y_train.shape, test_data.shape

((42000, 28, 28, 1), (42000, 1), (28000, 28, 28, 1))

In [124]:
X_train = X_train/255.0
test_data = test_data/255.0

In [125]:
Y_train = np_utils.to_categorical(Y_train)
Y_train.shape

(42000, 10)

In [126]:
def digit_rec_model(input_shape=(28, 28, 1), num_classes=10, lr=0.001):
    model = Sequential()
    
    model.add(Conv2D(32,(5,5), input_shape=input_shape, activation='relu', bias_initializer='RandomNormal'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    
    model.add(Conv2D(64,(5,5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    
    model.add(Dropout(0.5))
    model.add(Flatten())
    
    model.add(Dense(25, activation='relu'))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer = RMSprop(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [127]:
digit_rec_model = digit_rec_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_11 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 64)          51264     
_________________________________________________________________
batch_normalization_12 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 4, 4, 64)          0         
__________

In [128]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, 
                                                  test_size=0.1, random_state=83)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((37800, 28, 28, 1), (4200, 28, 28, 1), (37800, 10), (4200, 10))

In [135]:
digit_rec_model.fit(x_train,y_train, validation_data=(x_val, y_val), epochs=1,
                    batch_size=200, verbose=1)

Train on 37800 samples, validate on 4200 samples
Epoch 1/1
37800/37800 [==============================] - 81s - loss: 0.0204 - acc: 0.9935 - val_loss: 0.0557 - val_acc: 0.9862


In [133]:
y_pred = digit_rec_model.predict(test_data, batch_size=2000, verbose = 1)

28000/28000 [==============================] - 23s    


In [136]:
y_pred = np.argmax(y_pred,axis =1)

In [137]:
to_submit = pd.DataFrame({'Label': y_pred})
# Re-index to start at 1 instead of 0
to_submit.index += 1
to_submit.index.name = "ImageId"
to_submit.to_csv('./output/submit.csv')